<a href="https://colab.research.google.com/github/florianSari5/Google_colab/blob/master/fashion_mnist_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import & loading data

In [1]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D,AveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Concatenate
from keras import backend as K
import keras
import numpy as np

from keras import Model, Input
from keras.datasets import fashion_mnist
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

Using TensorFlow backend.


4423680/4422102 [==============================] - 2s 1us/step


## Pre-processing

In [0]:
batch_size = 128
num_classes = 10
epochs = 100

img_rows, img_cols = 28,28
chanDim = -1

In [0]:
if K.image_data_format() == 'channel_first':
  x_train = x_train.reshape(x_train.shape[0],1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0],1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0],img_rows, img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
    input_shape = (img_rows, img_cols, 1)

x_train =x_train.astype('float32')
x_test =x_test.astype('float32')
x_train/= 255
x_test/= 255


datagen = ImageDataGenerator(horizontal_flip=True)
datagen.fit(x_train, augment=True)



print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
#Model with shortcut
model_input= Input(shape=(28,28,1))

x =Activation("relu")(model_input)
x =BatchNormalization(axis=chanDim)(x)
conv1_output =Conv2D(32, (1, 1), padding="same")(x)

x =Activation("relu")(x)
x =BatchNormalization(axis=chanDim)(x)
x =MaxPooling2D(pool_size=(2, 2))(x)
x =Dropout(0.25)(x)

x =Conv2D(16, (1, 1), padding="same")(x)
x =Conv2D(16, (3, 3), padding="same")(x)
x =Activation("relu")(x)
x =BatchNormalization(axis=chanDim)(x)

x =MaxPooling2D(pool_size=(2, 2))(x)
x =Dropout(0.25)(x)

z = MaxPooling2D(pool_size=(4, 4))(conv1_output)
Concatenate()([x,z])

x =Flatten()(x)
x =Dense(200)(x)
x =Activation("relu")(x)
x =BatchNormalization()(x)
x =Dropout(0.5)(x)
x =Dense(num_classes)(x)
x =Activation("softmax")(x)

model = Model(inputs=[model_input], outputs=[x])

model.summary()
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=sgd,metrics=['accuracy'])


In [29]:
#model 94,00%
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(32, (1, 1), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (1, 1), padding="same"))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))


model.add(Conv2D(16, (1, 1), padding="same"))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(16, (1, 1), padding="same"))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.summary()
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=rmsprop,metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0)
modelCheckPoint = keras.callbacks.ModelCheckpoint('./model_94', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_105 (Conv2D)          (None, 28, 28, 64)        320       
_________________________________________________________________
activation_36 (Activation)   (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_92 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 28, 28, 32)        2080      
_________________________________________________________________
activation_37 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_93 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 14, 14, 32)      

In [0]:
#model 94,5%
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(16, (1, 1), padding="same"))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(16, (1, 1), padding="same"))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.1))
model.add(Dense(10, activation = "softmax"))

model.summary()
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=rmsprop,metrics=['accuracy'])
modelCheckPoint = keras.callbacks.ModelCheckpoint('./model_94', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0)


In [26]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://0bff972b.ngrok.io


In [0]:
#Data augmentation
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
	validation_data =(x_test, y_test), steps_per_epoch=len(x_train)//batch_size,
	epochs=epochs, shuffle=True,callbacks=[reduce_lr, modelCheckPoint, TensorBoardColabCallback(tbc)]) 
score=model.evaluate(x_test, y_test, verbose=0)

Epoch 1/150
468/468 [==============================] - 14s 29ms/step - loss: 0.6114 - acc: 0.7840 - val_loss: 0.5221 - val_acc: 0.8255

Epoch 00001: saving model to ./model_94
Epoch 2/150
468/468 [==============================] - 12s 26ms/step - loss: 0.3533 - acc: 0.8718 - val_loss: 0.3283 - val_acc: 0.8863

Epoch 00002: saving model to ./model_94
Epoch 3/150
468/468 [==============================] - 12s 26ms/step - loss: 0.3022 - acc: 0.8899 - val_loss: 0.3966 - val_acc: 0.8654

Epoch 00003: saving model to ./model_94
Epoch 4/150
468/468 [==============================] - 12s 26ms/step - loss: 0.2770 - acc: 0.8984 - val_loss: 0.2903 - val_acc: 0.8940

Epoch 00004: saving model to ./model_94
Epoch 5/150
468/468 [==============================] - 12s 27ms/step - loss: 0.2621 - acc: 0.9056 - val_loss: 0.2522 - val_acc: 0.9072

Epoch 00005: saving model to ./model_94
Epoch 6/150
468/468 [==============================] - 13s 27ms/step - loss: 0.2447 - acc: 0.9110 - val_loss: 0.2953 - v

In [0]:
model.save('fashion_mnist_model_.h5')

In [0]:
from keras import metrics
y_pred = model.predict(x_test)
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

## Printing

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    ax = plt.subplot(2, n , i+1)
    plt.imshow(x_test[110+i].reshape(28, 28))
    plt.gray()

plt.show()